In [48]:
print('hello')

hello


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle
from datetime import datetime


In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocessing the Data
## Drop the columns which are not required

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [5]:
from sklearn.preprocessing import OneHotEncoder

## Encoding the Categorical Data

le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])

## Encoding the Geography 
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_encoder = ohe.fit_transform(data[['Geography']])
geo_encoder.toarray()





## Splitting the data into features and target variable

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df =  pd.DataFrame(geo_encoder.toarray(), columns=ohe.get_feature_names_out(['Geography']))

In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## Save the encoders and scaler in pickle format

with open('encoders.pkl', 'wb') as f:
    pickle.dump(le, f)
    
with open('one hot encoders.pkl', 'wb') as f:
    pickle.dump(ohe, f)



In [11]:
## Divide the data into dependent and independent

X = data.drop('Exited', axis=1)
y = data['Exited']

## Splitting the data into training and testing set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature Scaling

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Save the scaler in pickle format 

with open('scaler.pkl', 'wb') as f:
    pickle.dump(sc, f)

## ANN Implementation

In [13]:
## Artificial neural network

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

model = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
## compile the training
import datetime

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
##Training the model

history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping, tensorflow_callback])

Epoch 1/50


250/250 [==============================] - 4s 8ms/step - loss: 0.4434 - accuracy: 0.8092 - val_loss: 0.3939 - val_accuracy: 0.8330
Epoch 2/50
250/250 [==============================] - 1s 5ms/step - loss: 0.3814 - accuracy: 0.8446 - val_loss: 0.3532 - val_accuracy: 0.8570
Epoch 3/50
250/250 [==============================] - 1s 6ms/step - loss: 0.3522 - accuracy: 0.8571 - val_loss: 0.3478 - val_accuracy: 0.8610
Epoch 4/50
250/250 [==============================] - 1s 5ms/step - loss: 0.3431 - accuracy: 0.8579 - val_loss: 0.3452 - val_accuracy: 0.8575
Epoch 5/50
250/250 [==============================] - 1s 5ms/step - loss: 0.3398 - accuracy: 0.8608 - val_loss: 0.3423 - val_accuracy: 0.8585
Epoch 6/50
250/250 [==============================] - 1s 6ms/step - loss: 0.3346 - accuracy: 0.8625 - val_loss: 0.3425 - val_accuracy: 0.8630
Epoch 7/50
250/250 [==============================] - 1s 5ms/step - loss: 0.3329 - accuracy: 0.8659 - val_loss: 0.3433 - val_accuracy: 0.8610
Epoc

In [23]:
model.save('model.h5')

d:\Udemy ML\projects\Deep Learning\ann\ann\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
## Load Tensorboard Extension

%load_ext tensorboard


In [29]:
%tensorboard --logdir logs/fit/20240823-091556

Reusing TensorBoard on port 6006 (pid 22996), started 0:06:03 ago. (Use '!kill 22996' to kill it.)